In [1]:
#load package
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import math
import time
from datetime import datetime
import copy
import numpy as np
import collections
import seaborn as sns
from scipy.spatial import distance_matrix
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.progress import *

In [2]:
import os
os.getcwd()

'C:\\Users\\Zhifeng Guo\\python_code\\paper3_code\\Benchmark'

In [3]:
def find_most_closed_points(dij,k):
    N=dij.shape[0]
    min_index=[]
    for i in range(N-k,N):
        mini_value=math.inf
        for j in range(N-k):
            if dij[i,j]<mini_value:
                mini_value=dij[i,j]
                mini_index=j
        min_index.append(mini_index)
    return min_index

def find_most_r_closed_points(dij,k,r):
    N=dij.shape[0]
    min_index=[]
    for i in range(N-k,N):
        row=dij.iloc[i,0:N-k]
        col_1=[col for col in row.nsmallest(n=r, keep='first').index]
        min_index.append(col_1)
    return min_index

def get_y_values(y):
    Yj=[]
    for i in M:
        if y[i].solution_value==1:
            Yj.append(i)
    return Yj

def init_lambdas(N,ddfij):
    lambda0=ddfij.mean().mean()
    #lambda0=ddfij.mean().mean()/p
    lambdas=np.zeros(len(N))
    lambdas.fill(lambda0)
    return lambdas

def LR_lower_solver(name, lambdas, N, M, ML, CL, p, ddfij):
    mdl=Model(name=name)
    ddfij_1=copy.deepcopy(ddfij)
    
    ddfij_2=ddfij_1.to_numpy()
    #set variables
    DL=ddfij_2-lambdas[:, None]

    x=mdl.binary_var_dict([(i,j) for i in N for j in M],lb=0.0, name="x")
    y=mdl.binary_var_dict([j for j in M],name="y")
    
    mdl.add_constraints(x[i, j] -y[j]<= 0 for i in N for j in M)

    mdl.add_constraints(x[row[1], j]+x[row[2], j] -1<= 0  for row in CL.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]+x[row[1], j] -1<= 0  for row in CL.itertuples() for j in M)

    mdl.add_constraints(x[row[1], j]-x[row[2], j] == 0  for row in ML.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]-x[row[1], j] == 0  for row in ML.itertuples() for j in M)

    mdl.add_constraint(mdl.sum(y[j] for j in M) == p)
    
    mdl.minimize(mdl.sum(DL[i, j]*x[i,j] for i in N for j in M))
    sol_model = mdl.solve(clean_before_solve=True)
    #print("Lower solved!")
    z_FB1=sol_model.get_objective_value()
    z_FB2=lambdas.sum()
    z_FB=z_FB1+z_FB2
    #print(f"LB1:{z_FB1}")
    #print(f"LB_lambda:{z_FB2}")
    print(f"LB:{z_FB}")

    L_Xij=np.zeros(shape=(len(N),len(M)))
    L_Yj=np.zeros(shape=(len(M)))
    for i in N:
        for j in M:
            L_Xij[i,j]=x[(i,j)].solution_value
    for j in M:
        L_Yj[j]=y[j].solution_value
    
    return z_FB, L_Xij, L_Yj


#获取super_point相关列表
def get_super_point_list(ML):
    position_list=[]
    super_list=[]
    super_list_double=[]
    for index,row in ML.iterrows():
        position_list.append(index)
        f1,f2 = row['f1'],row['f2']
        super_list.append(f1)
        super_list.append(f2)
        super_list_double.append([f1,f2])
    return position_list, super_list, super_list_double

def add_super_points(ML, ddfij,column1):
    ddfij_1=copy.deepcopy(ddfij)
    ddfij_2=ddfij_1.to_numpy()
    #rdij = pd.DataFrame(np.zeros((0, ddfij.shape[1])))
    appended_data = []
    for index,row in ML.iterrows():
        #print(index)
        f1,f2 = row['f1'],row['f2']
        #print(f1)
        must1=ddfij_2[f1,:]
        must2=ddfij_2[f2,:]
        must_sum=must1+must2
        must_df=pd.DataFrame(must_sum.reshape(1,-1), columns=column1)
        appended_data.append(must_df)
    super_dij = pd.concat(appended_data,ignore_index=True)
    return super_dij

#更新CLlist
def add_cannot_points(CL, super_list, super_name_list):
    CL_list=[]
    s_cannot_list=[]
    
    for index, row in CL.iterrows():
        f1,f2 = row['f1'],row['f2'] 
        pair=[]
        #print(index)
        #print(f1)
        #print(f2)
        if f1 in super_list:
            index = int(np.floor(super_list.index(f1)/2))
            pair.append(super_name_list[index])
            s_cannot_list.append(super_name_list[index])
            #print('add f1 from super point')                  
        else:
            #print('f1 is NOT in common_point_set')
            pair.append(f1)
            s_cannot_list.append(f1)
            #print('add f1 from normal point')
            
        if f2 in super_list:
            #print('f2 is in common_point_set')
            index = int(np.floor(super_list.index(f2)/2))
            pair.append(super_name_list[index])
            s_cannot_list.append(super_name_list[index])
            #print('add f2 from super point')
                                        
        else:
            #print('f2 is NOT in common_point_set')
            pair.append(f2)
            s_cannot_list.append(f2)
            #print('add f2 from super point')
        CL_list.append(pair)
    return  CL_list,s_cannot_list

def get_final_CL_list(CL_list):
    remain_part=copy.deepcopy(CL_list)
    final_cl_list=[]
    while remain_part !=[]: 
        a=remain_part[0]
        #print(len(remain_part))
        while (set(a) & (set().union(*remain_part))!=set()):
            #print(set(a) & (set().union(*remain_part)))
            #print(len(a))
            for b in remain_part:
                if list(set(a) & set(b))!=[]:
                    a+=b
                    remain_part.remove(b)
        final_cl_list.append(list(set(a)))
    return final_cl_list

def generate_cannot_point_dict(CL_list):
    CL_points=set().union(*CL_list)
    CL_dict={}
    for ele in CL_points:
        values=[]
        for cl_pair in CL_list:
            if ele in cl_pair:
                values.append(cl_pair)
        cannot_points_set=set().union(*values)
        #cannot_points_set.remove(ele)
        CL_dict.update({ele: list(cannot_points_set)})
    return CL_dict

def generate_cannot_point_dict_1(CL_list):
    CL_points=set().union(*CL_list)
    CL_dict={}
    for ele in CL_points:
        values=[]
        for cl_pair in CL_list:
            if ele in cl_pair:
                values.append(cl_pair)
        cannot_points_set=set().union(*values)
        cannot_points_set.remove(ele)
        CL_dict.update({ele: list(cannot_points_set)})
    return CL_dict

def use_CPLEX(row_names, col_names, small_dij, part_2_CL_dict):
    mdl=Model(name="small_CPLEX Model")
    x=mdl.binary_var_dict([(i,j) for i in row_names for j in col_names],lb=0.0, name="x")
    #print(x)
    mdl.add_constraints(mdl.sum(x[i, j] for j in col_names) == 1 for i in row_names)
    for p1 in part_2_CL_dict:
        for p2 in part_2_CL_dict[p1]:
            mdl.add_constraints(x[p1,j]+x[p2, j] -1<= 0  for j in col_names)
    mdl.minimize(mdl.sum(small_dij.loc[i, j]*x[i,j] for i in row_names for j in col_names))   
    sol_model = mdl.solve(clean_before_solve=True)
    three_above_obj=sol_model.get_objective_value()
    
    adjust_list=[]
    for i in row_names:
        for j in col_names:
            if x[(i,j)].solution_value==1:
                adjust_list.append([i,j])
                
    return adjust_list, three_above_obj

def feasible_solution(reduced_dij, CL_list, CL_dict, column1,L_Yj):
    #提取对应的L_Yj的column
    violated_pairs=[]
    reduced_1_dij=copy.deepcopy(reduced_dij)
    
    reduced_2_dij=reduced_1_dij.drop(['id'], axis=1)
    
    #solution_dij=reduced_dij.iloc[:,0:len(column1)]
    
    bool_col=np.array(L_Yj, dtype=bool)
    
    reduced_3_dij=reduced_2_dij.loc[:,bool_col]
    
    
    solution_dij=copy.deepcopy(reduced_3_dij)
    solution_dij1=copy.deepcopy(reduced_3_dij)
    solution_dij2=copy.deepcopy(reduced_3_dij)
    
    #col_names=solution_dij.columns
    
    
    #solution_dij1=solution_dij.loc[:,bool_col]
    #solution_dij2=solution_dij.loc[:,bool_col]
    #print(solution_dij2.columns)
    solution_dij2['id']=reduced_dij['id']
    
        
    row_min=solution_dij1.min(axis=1)
    
    boolean_sol=solution_dij1.sub(row_min,axis='index')
    boolean_sol1= boolean_sol.mask(boolean_sol  == 0, 0.0000001)

    boolean_sol2=boolean_sol1.mask(boolean_sol1 >0.0000001, 0)
    boolean_sol3=boolean_sol2.mask(boolean_sol2 ==0.0000001, 1)
    
    boolean_sol3.index=reduced_dij['id']
    y=boolean_sol3.idxmax(axis=1)

    
    opt_obj=solution_dij1.min(axis=1).sum()
    
    #print(f"opt_obj:{opt_obj}")
    #返回每一行的最小值

    #检查冲突对儿,获取位置
    #move_number=0
    moved_points=[]
    
    for pair_list in CL_list:
        p1=pair_list[0]
        p2=pair_list[1]
        if y[p1] == y[p2]:
            violated_pairs.append(pair_list)
    
    
    reduced_6_dij=copy.deepcopy(reduced_3_dij)
    reduced_6_dij['y']=y.values
    reduced_6_dij['id']=reduced_1_dij['id']
    
    
    for pair_list in violated_pairs:      
        p1=pair_list[0]
        p2=pair_list[1]
        
        if p1 not in moved_points and p2 not in moved_points: 
            p1_cannot_points=CL_dict[p1]
            p2_cannot_points=CL_dict[p2]
            p1_cannot_centers=[]
            p2_cannot_centers=[]

            for point1 in p1_cannot_points:
                y_p1_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point1,"y"].values[0]
                p1_cannot_centers.append(y_p1_cannot)

            for point2 in p2_cannot_points:
                y_p2_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point2,"y"].values[0]
                p2_cannot_centers.append(y_p2_cannot)


            row1=reduced_6_dij[reduced_6_dij['id']==p1].iloc[:,0:int(sum(L_Yj))]
            row1_remove_cannot=row1[[c for c in row1.columns if c not in p1_cannot_centers]]
            could_not_move_p1=[c for c in row1.columns if c not in p1_cannot_centers]==[]

            row2=reduced_6_dij[reduced_6_dij['id']==p2].iloc[:,0:int(sum(L_Yj))]
            row2_remove_cannot=row2[[c for c in row2.columns if c not in p2_cannot_centers]]
            could_not_move_p2=[c for c in row2.columns if c not in p2_cannot_centers]==[]

            cost1 = math.inf if could_not_move_p1 else row1_remove_cannot.values.min()-row1.values.min()
            cost2 = math.inf if could_not_move_p2 else row2_remove_cannot.values.min()-row2.values.min()
                #print(f"cost1:{cost1}")
                #print(f"cost2:{cost2}")


            if cost1==math.inf and cost2==math.inf:
                print("Feasibility cannot be restored!!!")
                print(f"Could_not_move_p1:{could_not_move_p1}")
                print(f"Could_not_move_p2:{could_not_move_p2}")

            elif cost1<= cost2:
                    #print(f"moving:{p1}")
                opt_obj += cost1
                    #print(f"p1:{cost1}")
                moved_points.append(p1)
                p1_closest_center=row1.idxmin(axis=1).values[0]
                p1_second_closest_center=row1_remove_cannot.idxmin(axis=1).values[0]
                boolean_sol3.loc[p1, p1_closest_center] = 0
                boolean_sol3.loc[p1, p1_second_closest_center] = 1

            else:
                    #print(f"moving:{p2}")
                opt_obj += cost2 
                moved_points.append(p2)
                   # print(f"p2:{cost2}")
                p2_closest_center=row2.idxmin(axis=1).values[0]
                p2_second_closest_center=row2_remove_cannot.idxmin(axis=1).values[0]
                boolean_sol3.loc[p2, p2_closest_center] = 0
                boolean_sol3.loc[p2, p2_second_closest_center] = 1

        #print(f"move number: {move_number}")
        #print(f"updated opt:{opt_obj}")
    return opt_obj

def feasible_solution_part_cplex(reduced_dij, final_cl_list, CL_dict, CL_dict1,column1,L_Yj):
    #提取对应的L_Yj的column
    reduced_1_dij=copy.deepcopy(reduced_dij)
    
    #reduced_1_dij.index=reduced_1_dij['id']
    reduced_2_dij=reduced_1_dij.drop(['id'], axis=1)

    #reduced_3_dij=reduced_1_dij.iloc[:,0:int(sum(L_Yj))]
    bool_col=np.array(L_Yj, dtype=bool)
    #print(len(bool_col))
    #reduced_2_dij索引为id
    
    reduced_3_dij=reduced_2_dij.loc[:,bool_col]
    solution_dij=copy.deepcopy(reduced_3_dij)

    
    #solution_dij2['id']=reduced_dij['id']
    
    col_names=solution_dij.columns
    #print(f"col_names:{col_names}")
    row_min=solution_dij.min(axis=1)
    
    boolean_sol=solution_dij.sub(row_min,axis='index')
    boolean_sol1= boolean_sol.mask(boolean_sol  == 0, 0.0000001)

    boolean_sol2=boolean_sol1.mask(boolean_sol1 > 0.0000001, 0)
    boolean_sol3=boolean_sol2.mask(boolean_sol2 ==0.0000001, 1)
    
        
    #boolean_sol3.index=reduced_dij['id']

    three_above_points=[item for item in final_cl_list if len(item) >= 3]
    
    part_2_points=list(set().union(*three_above_points))
    
    reduced_4_dij=copy.deepcopy(reduced_dij)
    
    reduced_dij_part_1=reduced_4_dij.loc[~reduced_4_dij['id'].isin(part_2_points)]
    reduced_dij_part_2=reduced_dij_part_1.drop(['id'], axis=1)
    reduced_dij_part_3=reduced_dij_part_2.loc[:,bool_col]
  
    opt_obj_part_1=reduced_dij_part_3.min(axis=1).sum()
    
    #opt_obj=reduced_1_dij.min(axis=1).sum()
    
    
    #print(f"opt_obj_part_1:{opt_obj_part_1}")
    #print(f"opt_obj_part_2:{opt_obj_part_2}")
    
    
    for pair_list in final_cl_list:
        if len(pair_list)==2:
            p1=pair_list[0]
            p2=pair_list[1]
            #print(reduced_3_dij.columns)
            reduced_6_dij=copy.deepcopy(reduced_3_dij)

            y=reduced_6_dij.idxmin(axis=1)
            
            reduced_6_dij['y']=y.values
            reduced_6_dij['id']=reduced_1_dij['id']
            
            row1=reduced_6_dij[reduced_6_dij['id']==p1]
            y1=row1['y'].values[0]
            
            row2=reduced_6_dij[reduced_6_dij['id']==p2]
            y2=row2['y'].values[0]


            #print(f"y1:{y1}")
            #print(f"y2:{y2}")

            if y1==y2:
                #print(f"p1:{p1}")
                #print(f"p2:{p2}")
                p1_cannot_points=CL_dict[p1]
                p2_cannot_points=CL_dict[p2]
                p1_cannot_centers=[]
                p2_cannot_centers=[]
                
                for point1 in p1_cannot_points:
                    y_p1_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point1,"y"].values[0]
                    p1_cannot_centers.append(y_p1_cannot)

                for point2 in p2_cannot_points:
                    y_p2_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point2,"y"].values[0]
                    p2_cannot_centers.append(y_p2_cannot)


                row1=reduced_6_dij[reduced_6_dij['id']==p1].iloc[:,0:int(sum(L_Yj))]
                row1_remove_cannot=row1[[c for c in row1.columns if c not in p1_cannot_centers]]
                could_not_move_p1=[c for c in row1.columns if c not in p1_cannot_centers]==[]
                
                row2=reduced_6_dij[reduced_6_dij['id']==p2].iloc[:,0:int(sum(L_Yj))]
                row2_remove_cannot=row2[[c for c in row2.columns if c not in p2_cannot_centers]]
                could_not_move_p2=[c for c in row2.columns if c not in p2_cannot_centers]==[]

                cost1 = math.inf if could_not_move_p1 else row1_remove_cannot.values.min()-row1.values.min()
                cost2 = math.inf if could_not_move_p2 else row2_remove_cannot.values.min()-row2.values.min()
                #print(f"cost1:{cost1}")
                #print(f"cost2:{cost2}")
                

                if cost1==math.inf and cost2==math.inf:
                    print("Feasibility cannot be restored!!!")
                    #print(f"Could_not_move_p1:{could_not_move_p1}")
                    #print(f"Could_not_move_p2:{could_not_move_p2}")

                elif cost1<= cost2:
                    #print('move p1')
                    #print(f"moving:{p1}")
                    opt_obj_part_1 += cost1
                    #print(f"p1:{cost1}")
                
                    p1_closest_center=row1.idxmin(axis=1).values[0]
                    p1_second_closest_center=row1_remove_cannot.idxmin(axis=1).values[0]
                    boolean_sol3.loc[p1, p1_closest_center] = 0
                    boolean_sol3.loc[p1, p1_second_closest_center] = 1

                else:
                    #print('move p2')
                    #print(f"moving:{p2}")
                    opt_obj_part_1 += cost2 
                    #print(f"p2:{cost2}")
                    p2_closest_center=row2.idxmin(axis=1).values[0]
                    p2_second_closest_center=row2_remove_cannot.idxmin(axis=1).values[0]
                    boolean_sol3.loc[p2, p2_closest_center] = 0
                    boolean_sol3.loc[p2, p2_second_closest_center] = 1
            else:
                #print("no need to move any point")
                opt_obj_part_1 += 0

                
        else:
            #print("Three or more points, We plan to use CPLEX")
            row_names=pair_list
            
            part_2_CL_dict={}
            for ele in row_names:
                part_2_constraints=CL_dict1[ele]
                part_2_CL_dict.update({ele: list(part_2_constraints)})
            
            reduced_5_dij=copy.deepcopy(reduced_dij)
            reduced_5_dij.index=reduced_5_dij['id']
            
            small_dij=reduced_5_dij.loc[row_names,col_names]
            
            adjust_list, three_above_obj=use_CPLEX(row_names, col_names,small_dij, part_2_CL_dict)
            #print("three_above_obj")
            #print(three_above_obj)
            for adj in adjust_list:
                boolean_sol3.loc[adj[0], :] = 0
                boolean_sol3.loc[adj[0], adj[1]] = 1
            
            
            #print(f"three_above_obj:{three_above_obj}")
            
            opt_obj_part_1+=three_above_obj
        
    #print(f"move number: {move_number}")
    #print(f"updated opt:{opt_obj_part_1}")
    return opt_obj_part_1

def revised_feasible_solution(reduced_dij, CL_list, CL_dict, column1,L_Yj):
    violated_pairs=[]
    point_Dict = {}
    
    for i in reduced_dij['id'].values:
        point_Dict[i]=[]
    #每个字典中的元素初始化为空的list[]
    
    #generate n empty dict
    
    #提取对应的L_Yj的column
    reduced_1_dij=copy.deepcopy(reduced_dij)
    
    reduced_2_dij=reduced_1_dij.drop(['id'], axis=1)
    
    #solution_dij=reduced_dij.iloc[:,0:len(column1)]
    
    bool_col=np.array(L_Yj, dtype=bool)
    
    reduced_3_dij=reduced_2_dij.loc[:,bool_col]
    
    
    solution_dij=copy.deepcopy(reduced_3_dij)
    solution_dij1=copy.deepcopy(reduced_3_dij)
    solution_dij2=copy.deepcopy(reduced_3_dij)
    
    #col_names=solution_dij.columns
    
    
    #solution_dij1=solution_dij.loc[:,bool_col]
    #solution_dij2=solution_dij.loc[:,bool_col]
    #print(solution_dij2.columns)
    solution_dij2['id']=reduced_dij['id']
    
        
    row_min=solution_dij1.min(axis=1)
    
    boolean_sol=solution_dij1.sub(row_min,axis='index')
    boolean_sol1= boolean_sol.mask(boolean_sol  == 0, 0.0000001)

    boolean_sol2=boolean_sol1.mask(boolean_sol1 >0.0000001, 0)
    boolean_sol3=boolean_sol2.mask(boolean_sol2 ==0.0000001, 1)
    
   
    
    boolean_sol3.index=reduced_dij['id']
    y=boolean_sol3.idxmax(axis=1)

    
    opt_obj=solution_dij1.min(axis=1).sum()
    
    #print(f"opt_obj:{opt_obj}")
    #返回每一行的最小值

    #检查冲突对儿,获取位置
    #move_number=0
    for pair_list in CL_list:
        p1=pair_list[0]
        p2=pair_list[1]
        if y[p1] == y[p2]:
            point_Dict[p1].append(p2)
            point_Dict[p2].append(p1)
            violated_pairs.append(pair_list)
    #print(point_Dict)
    #print(violated_pairs)
    
    reduced_6_dij=copy.deepcopy(reduced_3_dij)
    reduced_6_dij['y']=y.values
    reduced_6_dij['id']=reduced_1_dij['id']
    moved_points=[]
    for pair_list in violated_pairs:
        #y=boolean_sol3.idxmax(axis=1)
        Wi=0
        Wj=0
        #print(violated_pairs)
        #print(f"moved_points:{moved_points}")
        #if pair_list not in 
        p11=pair_list[0]
        p22=pair_list[1]
        
        if p11 not in moved_points and p22 not in moved_points:    
        #print(f"p11:{p11}")
        #print(f"p22:{p22}")
            for point_1 in point_Dict[p11]:
                #print(f"if moving:{point_1}")
                p1_cannot_points=CL_dict[point_1]
                p1_cannot_centers=[]
                for point1_1 in p1_cannot_points:
                    y_p1_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point1_1,"y"].values[0]
                    p1_cannot_centers.append(y_p1_cannot)

                row1=reduced_6_dij[reduced_6_dij['id']==point_1].iloc[:,0:int(sum(L_Yj))]
                row1_remove_cannot=row1[[c for c in row1.columns if c not in p1_cannot_centers]]
                could_not_move_p1=[c for c in row1.columns if c not in p1_cannot_centers]==[]

                cost1 = math.inf if could_not_move_p1 else row1_remove_cannot.values.min()-row1.values.min()
                Wi += cost1      


            for point_2 in point_Dict[p22]:

                #print(f"IF moving:{point_2}")
                p2_cannot_points=CL_dict[point_2]
                p2_cannot_centers=[]      

                for point2_1 in p2_cannot_points:
                    y_p2_cannot=reduced_6_dij.loc[reduced_6_dij['id']==point2_1,"y"].values[0]
                    p2_cannot_centers.append(y_p2_cannot)

                row2=reduced_6_dij[reduced_6_dij['id']==point_2].iloc[:,0:int(sum(L_Yj))]
                row2_remove_cannot=row2[[c for c in row2.columns if c not in p2_cannot_centers]]
                could_not_move_p2=[c for c in row2.columns if c not in p2_cannot_centers]==[]

                cost2 = math.inf if could_not_move_p2 else row2_remove_cannot.values.min()-row2.values.min()
                Wj += cost2



            if Wi==math.inf and Wj==math.inf:
                print("Feasibility cannot be restored!!!")
                print(f"Could_not_move_p1:{could_not_move_p1}")
                print(f"Could_not_move_p2:{could_not_move_p2}")

            elif Wi<= Wj:
                #print(f"keeping:{p11}")
                opt_obj += Wi
                #print(f"Wi:{Wi}")
                #violated_pairs = remove_elements(violated_pairs,p11)

                for p_11 in point_Dict[p11]:
                    #print(f"moving:{p_11}")
                    moved_points.append(p_11)
                    row1=reduced_6_dij[reduced_6_dij['id']==p_11].iloc[:,0:int(sum(L_Yj))]
                    row1_remove_cannot=row1[[c for c in row1.columns if c not in p1_cannot_centers]]

                    p1_closest_center=row1.idxmin(axis=1).values[0]
                    p1_second_closest_center=row1_remove_cannot.idxmin(axis=1).values[0]
                    boolean_sol3.loc[p_11, p1_closest_center] = 0
                    boolean_sol3.loc[p_11, p1_second_closest_center] = 1

            else:
                #print(f"keeping:{p22}")
                opt_obj += Wj 
                #print(f"Wj:{Wj}")
                #violated_pairs = remove_elements(violated_pairs,p22)
                for p_22 in point_Dict[p22]:
                    #print(f"moving:{p_22}")
                    moved_points.append(p_22)
                    row2=reduced_6_dij[reduced_6_dij['id']==p_22].iloc[:,0:int(sum(L_Yj))]
                    row2_remove_cannot=row2[[c for c in row2.columns if c not in p2_cannot_centers]]

                    p2_closest_center=row2.idxmin(axis=1).values[0]
                    p2_second_closest_center=row2_remove_cannot.idxmin(axis=1).values[0]
                    boolean_sol3.loc[p_22, p2_closest_center] = 0
                    boolean_sol3.loc[p_22, p2_second_closest_center] = 1

    #print(f"move number: {move_number}")
    #print(f"updated opt:{opt_obj}")
    return opt_obj
    
    




def feasible_solution_only_cplex(name, N, M,ML,CL ,ddfij, L_Yj):
    mdl=Model(name=name)
    #set variables
    #fixed_y=1-L_Yj
    ddfij_3=copy.deepcopy(ddfij)
    
    ddfij_4=ddfij_3.to_numpy()
    x=mdl.binary_var_dict([(i,j) for i in N for j in M],lb=0.0, name="x")
    
    mdl.add_constraints(x[i, j] -L_Yj[j]<= 0 for i in N for j in M)
    #mdl.add_constraints(x[i, j] -fixed_y[j]<= 0 for i in N for j in M)
    
    mdl.add_constraints(mdl.sum(x[i, j] for j in M) == 1 for i in N)
    
    mdl.add_constraints(x[row[1], j]+x[row[2], j] -1<= 0  for row in CL.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]+x[row[1], j] -1<= 0  for row in CL.itertuples() for j in M)

    mdl.add_constraints(x[row[1], j]-x[row[2], j] == 0  for row in ML.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]-x[row[1], j] == 0  for row in ML.itertuples() for j in M)    

    
    mdl.minimize(mdl.sum(ddfij_4[i, j]*x[i,j] for i in N for j in M))
    
    sol_model = mdl.solve(clean_before_solve=True)
    print("feasible model solved!")
    z_FB=sol_model.get_objective_value()
    print(f"UB:{z_FB}")
    U_Xij=np.zeros(shape=(len(N),len(M)))
    for i in N:
        for j in M:
            U_Xij[i,j]=x[(i,j)].solution_value
    
    return z_FB, U_Xij,L_Yj

def subgradient_iteration(lambdas, subgradient,L_Xij, z_UB_best, z_LR,  epsilon, beta):    
    #if np.linalg.norm(subgradient)<0.002:
       # return lambdas
    alpha = beta * ( z_UB_best - z_LR )/(np.square(subgradient).sum())
    #print(f"beta_b:{( z_UB_best - z_LR )/(np.square(subgradient).sum())}")
    #alpha = beta * ( z_UB_best - z_LR )/(np.abs(subgradient).sum())
    #alpha = beta * ( z_UB_best - z_LR )/(L_Xij.sum()-n)
    #print(f"subgradient^2:{np.square(subgradient).sum()}")
    #print(f"alpha:{alpha}")
    #print(f"beta:{beta}")
    #print(alpha)
    for i in range(lambdas.shape[0]):
        if (np.abs(subgradient[i]) > epsilon):                
            lambdas[i] = max(lambdas[i] - alpha * subgradient[i], 0)
    #print(f" lambdas: {lambdas}")
    #print(f"step: {subgradient*alpha}")
    return lambdas


def xij_to_label(U_Xij):
    labels=[]
    for i in N:
        for j in M:
            if U_Xij[i,j]!=0:
                labels.append(j)
    return labels


In [4]:
def solve_lagrangian_dual(n, N,M,ML,CL, ddfij,reduced_dij,CL_list,CL_dict, column1,p,epsilon,beta,iterations,lambda0):
    z_LB=np.full(iterations, -np.inf)
    z_UB=np.full(iterations, np.inf)
    z_LR=np.zeros(iterations)
    z_FB=np.zeros(iterations)
    #z_Optimal=np.full(iterations+1, 10515)21450
    #z_Optimal=np.full(iterations+1, 21450)
    lambdas=init_lambdas(N,ddfij)
    _lambdas = [] 
    _lambdas.append(lambdas.copy())
    time1=[]
    labels_list=[]
    Y_list=[]
    betas=[]
    gaps=[]
    rediter=0
    
    for t in range(iterations):

        start11=datetime.now()
        z_LR[t], L_Xij, L_Yj = LR_lower_solver(name="LowerModel",lambdas=lambdas, N=N, M=M, ML=ML,CL=CL, p=p, ddfij=ddfij)
        
        #z_FB[t], U_Xij, L_Yj = feasible_solution_only_cplex(name="UpperModel", N=N, M=M, ML=ML,CL=CL,ddfij=ddfij, L_Yj=L_Yj)
        #z_FB[t]= feasible_solution(reduced_dij,CL_list,CL_dict,column1,L_Yj)
        #z_FB[t]=feasible_solution_part_cplex(reduced_dij, final_cl_list, CL_dict, CL_dict1,column1,L_Yj)
        #print(f"upper bound by CPLEX:{z_FB_CPLEX}")
        z_FB[t]= revised_feasible_solution(reduced_dij, CL_list, CL_dict, column1,L_Yj)
        
        
        z_UB[t]= min(z_FB[t], z_UB[t])
        z_UB_best=min(z_UB)
        z_LB[t] = max(z_LB[t], z_LR[t])
        z_LB_best=max(z_LB)
        gap=np.abs(z_UB_best-z_LB_best)/np.abs(z_LB_best)
        #gap=np.abs(z_UB_best-260.509742450219)/np.abs(260.509742450219)
        
        gaps.append(gap)
        print(f"gap:{gap}")
        
        if t>=2 and z_LR[t]<z_LR[t-1]:
            rediter=rediter+1
        
        #if t>=2 and (gaps[t]>gaps[t-1]-0.00001) :
            #rediter=rediter+1
            
        #Objective does not decrease for 3 consecutive iterations
        if rediter == 2:
            beta = beta/2
            rediter = 1;
        
        subgradient = L_Xij.sum(axis=1)-1
        #subgradient = L_Xij.sum()-n
        #print(z_UB_best)
        lambdas=subgradient_iteration(lambdas, subgradient,L_Xij, z_UB_best, z_LR[t],epsilon, beta)
        #if t % math.ceil(iterations/2) == 0:
           # beta=beta/2
        #if t % math.ceil(iterations/3) == 0:
            #beta=beta/3
        timerecord=datetime.now()-start11
        print(f"upper bound by Heuristic:{z_FB[t]}")
        betas.append(beta)
        time1.append(timerecord)
        Y_list.append(L_Yj)
        labels=xij_to_label(L_Xij)
        labels_list.append(labels)
        #pd.DataFrame(L_Xij).to_csv('L_Xij' + str(t) + '.csv')
        #pd.DataFrame(U_Xij).to_csv('U_Xij' + str(t) + '.csv')
        #pd.DataFrame(L_Yj).to_csv('L_Yj' + str(t) + '.csv')

        _lambdas.append(lambdas.copy())

        print(t)
        #print(f"z_FB_best:{z_FB[t]}") 
        #print(f"z_LR_best:{z_LR[t]}")       
        if beta <=0.002 or gap<=0.0001:
            return z_FB[:t+1],z_UB[:t+1],z_LB[:t+1],z_LR[:t+1],_lambdas,time1,labels_list, labels,Y_list,betas, gaps
    
    return z_FB[:t+1],z_UB[:t+1],z_LB[:t+1],z_LR[:t+1],_lambdas,time1,labels_list, labels,Y_list,betas, gaps 

In [5]:
file_list=[['bc_datasets/Soybean/soybean.classes.txt',
            'bc_datasets/Soybean/soybean.data.txt',
            ['bc_datasets/Soybean/soybean1.txt','bc_datasets/Soybean/soybean2.txt']],
           
           ['bc_datasets/Protein/protein.classes.txt',
            'bc_datasets/Protein/protein.data.txt',
            ['bc_datasets/Protein/protein1.txt','bc_datasets/Protein/protein2.txt']],
           
           ['bc_datasets/Iris/iris.classes.txt',
            'bc_datasets/Iris/iris.data.txt',
            ['bc_datasets/Iris/iris1.txt','bc_datasets/Iris/iris2.txt']],
           
           ['bc_datasets/Wine/wine.classes.txt',
            'bc_datasets/Wine/wine.data.txt',
            ['bc_datasets/Wine/wine1.txt','bc_datasets/Wine/wine2.txt']],
           
           ['bc_datasets/Ionosphere/ionosphere.classes.txt',
            'bc_datasets/Ionosphere/ionosphere.data.txt',
            ['bc_datasets/Ionosphere/ionosphere1.txt','bc_datasets/Ionosphere/ionosphere2.txt']],
           
           ['bc_datasets/Control/control.classes.txt',
            'bc_datasets/Control/control.data.txt',
            ['bc_datasets/Control/control1.txt','bc_datasets/Control/control2.txt']],
           
           ['bc_datasets/Balance/balance.classes.txt',
            'bc_datasets/Balance/balance.data.txt',
            ['bc_datasets/Balance/balance1.txt','bc_datasets/Balance/balance2.txt']],
           
           ['bc_datasets/Yeast/yeast.classes.txt',
            'bc_datasets/Yeast/yeast.data.txt',
            ['bc_datasets/Yeast/yeast1.txt','bc_datasets/Yeast/yeast2.txt']]
          ]

In [10]:
results=[]
for files in file_list:
    label_df1=pd.read_csv(files[0],sep=" ",header=None)
    sampled_df1=pd.read_csv(files[1],sep=" ",header=None)
    p=label_df1.value_counts().shape[0]
    sampled_df1.columns = [f'X{i+1}' for i in range(len(sampled_df1.columns))]
    sampled_df1=sampled_df1.dropna(axis=1, how='any')
    df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]
    n=df_X.shape[0]
    configs=files[2]
    for config in configs:
        print(files[0])
        print(config)
        #print(label_df1.head())
        #print(sampled_df1.head())
        Constraints_data=pd.read_csv(config,sep="\t",header=None)
        SV1=[2*p]
        SV2=[5*p]
        SV3=[10*p]
        
        MV1=[p, p+1, p+2, p+3, p+4]
        MV2=[p, p+math.floor(p/2), p+2*math.floor(p/2)]
        MV3=[p, 2*p, 3*p]
        
        RN=[p]
        
        all_columns=[]
        for k in MV2:
            print(k)
            kmeans = KMeans(n_clusters=k, random_state=0).fit(df_X)
            cluster_centers=pd.DataFrame(kmeans.cluster_centers_,columns=df_X.columns, index=["center"+str(i) for i in range(k)])
            frames = [df_X, cluster_centers]
            df_X_1 = pd.concat(frames)
            dfij=pd.DataFrame(distance_matrix(df_X_1.values, df_X_1.values), index=df_X_1.index, columns=df_X_1.index)
            #column=list(find_most_r_closed_points(dfij,k,r=4))
            dij=dfij.to_numpy()
            column=list(set(find_most_closed_points(dij,k)))
            all_columns.append(column)
            #all_columns=all_columns[0]
        
        column1=list(set().union(*all_columns))
        N=range(sampled_df1.shape[0])
        M=range(len(set().union(*all_columns)))
        Constraints_data.columns = [f'f{i+1}' for i in range(len(Constraints_data.columns))]
        Constraints_data.head()
        ML=Constraints_data[Constraints_data['f3']==1]
        ML = ML.reset_index(drop=True)
        CL=Constraints_data[Constraints_data['f3']==-1]
        ddfij=dfij.iloc[0:n,column1]
        position_list, super_list, super_list_double=get_super_point_list(ML)
        super_name_list=[100000+i for i in range(len(super_list_double))]
        super_dij=add_super_points(ML,ddfij,column1)
        super_dij['id']=super_name_list
        ddfij['id']=ddfij.index
        reduced_super_dij=ddfij.drop(super_list)
        reduced_super_dij.columns=super_dij.columns
        reduced_dij=pd.concat([reduced_super_dij, super_dij],ignore_index=True)
        CL_list,s_cannot_list=add_cannot_points(CL, super_list, super_name_list)
        final_cl_list=get_final_CL_list(CL_list)
        CL_dict=generate_cannot_point_dict(CL_list)
        CL_dict1=generate_cannot_point_dict_1(CL_list)
        ddfij=dfij.iloc[0:n,column1]
        start2=datetime.now()
        z_FB, z_UB, z_LB,z_LR,_lambdas,time1,labels_list,labels,Y_list,betas,gaps=solve_lagrangian_dual(n=n,N=N,M=M,ML=ML, CL=CL, ddfij=ddfij,reduced_dij=reduced_dij,CL_list=CL_list,CL_dict=CL_dict,column1=column1,p=p,epsilon=0.001, beta=3,iterations=200,lambda0=200)
        running_time=datetime.now()-start2
        print(running_time)
        best_LB=max(z_LB)
        best_UB=min(z_UB)
        iterations=len(z_LB)
        result_each=[best_LB, best_UB, iterations, running_time]
        results.append(result_each)

             

bc_datasets/Soybean/soybean.classes.txt
bc_datasets/Soybean/soybean1.txt
4
6
8
LB:58.41235552384373
gap:1.8618372609262226
upper bound by Heuristic:167.16665553660565
0
LB:30.81677294370681
gap:1.217456052932489
upper bound by Heuristic:129.52683132239179
1
LB:70.0574803589122
gap:0.8488651127447282
upper bound by Heuristic:177.84351483245305
2
LB:43.99431222637759
gap:0.8488651127447282
upper bound by Heuristic:146.44045909651734
3
LB:72.68127030468594
gap:0.7821211816937778
upper bound by Heuristic:148.5209481969245
4
LB:45.35663667979901
gap:0.7821211816937778
upper bound by Heuristic:152.44089514965242
5
LB:86.69020139823681
gap:0.43529957350538573
upper bound by Heuristic:124.42640909398528
6
LB:81.23249606874695
gap:0.43529957350538573
upper bound by Heuristic:143.35267241556699
7
LB:101.91604868204763
gap:0.22087159679987498
upper bound by Heuristic:129.52683132239179
8
LB:103.90053250336632
gap:0.19755314141391853
upper bound by Heuristic:146.07327290998816
9
LB:112.33150423262

LB:1259.3065190803777
gap:0.031258499975112576
upper bound by Heuristic:1372.0775289594658
22
LB:1262.9420997916402
gap:0.02938232080022077
upper bound by Heuristic:1318.6029048857795
23
LB:1263.1022875119215
gap:0.029251773647470614
upper bound by Heuristic:1300.0502697198226
24
LB:1264.1572328330076
gap:0.006239743427901422
upper bound by Heuristic:1272.0452496184114
25
LB:1264.6351945625106
gap:0.0058594407998143905
upper bound by Heuristic:1318.6029048857795
26
LB:1265.2336380061151
gap:0.005383678877705688
upper bound by Heuristic:1272.0452496184114
27
LB:1265.6681476020728
gap:0.003604882082247085
upper bound by Heuristic:1270.2307320294344
28
LB:1266.1514582342502
gap:0.003221789754026039
upper bound by Heuristic:1288.4999594093244
29
LB:1266.5200382137846
gap:0.0029298342731972583
upper bound by Heuristic:1270.2307320294344
30
LB:1266.5818696947622
gap:0.002880873650553345
upper bound by Heuristic:1278.1234636932727
31
LB:1266.9393631271014
gap:0.0025978898423434952
upper bound

LB:1268.2169787051014
gap:0.001160077421192734
upper bound by Heuristic:1276.6983768548614
49
LB:1268.220612821871
gap:0.0011584899725964278
upper bound by Heuristic:1276.6983768548614
50
LB:1268.2209388343867
gap:0.0011582326119206347
upper bound by Heuristic:1269.6898336848653
51
LB:1268.220359243261
gap:0.0011582326119206347
upper bound by Heuristic:1277.9665771712089
52
0:00:06.893703
bc_datasets/Iris/iris.classes.txt
bc_datasets/Iris/iris1.txt
3
4
5
LB:-26.115831434382585
gap:10.388642641842399
upper bound by Heuristic:245.19220863201244
0
LB:-92.96348182077816
gap:5.844559274373298
upper bound by Heuristic:126.51969338340784
1
LB:-104.03785591815023
gap:5.844559274373298
upper bound by Heuristic:245.19220863201244
2
LB:35.20900879652565
gap:2.5933898086870464
upper bound by Heuristic:148.3948723994463
3
LB:-138.8359442291249
gap:2.5933898086870464
upper bound by Heuristic:245.19220863201244
4
LB:90.61557662217083
gap:0.39622455762702036
upper bound by Heuristic:126.51969338340784

LB:17723.406904914315
gap:0.005970679357495352
upper bound by Heuristic:17829.227684665977
33
LB:17724.02694854567
gap:0.005935487258381707
upper bound by Heuristic:17829.227684665977
34
LB:17724.64335910888
gap:0.005900503803556142
upper bound by Heuristic:17829.227684665977
35
LB:17725.256157891436
gap:0.005865727741725888
upper bound by Heuristic:17829.227684665977
36
LB:17725.865366056132
gap:0.005831157829268909
upper bound by Heuristic:17829.227684665977
37
LB:17726.471004641735
gap:0.005796792830187966
upper bound by Heuristic:17829.227684665977
38
LB:17727.073094563755
gap:0.005762631516059407
upper bound by Heuristic:17829.227684665977
39
LB:17727.671656615137
gap:0.0057286726659867854
upper bound by Heuristic:17829.227684665977
40
LB:17728.266711466997
gap:0.0056949150665517
upper bound by Heuristic:17829.227684665977
41
LB:17728.858279669334
gap:0.0056613575117661465
upper bound by Heuristic:17829.227684665977
42
LB:17729.446381651735
gap:0.005627998803025596
upper bound by 

LB:17757.604471777635
gap:0.004033382599672929
upper bound by Heuristic:17969.222877935514
121
LB:17757.85246996784
gap:0.004019360720495275
upper bound by Heuristic:17829.227684665977
122
LB:17758.107781870374
gap:0.004004925731344557
upper bound by Heuristic:17969.222877935514
123
LB:17758.29672900274
gap:0.003994243183660294
upper bound by Heuristic:17829.227684665977
124
LB:17758.607622264488
gap:0.003976666634210144
upper bound by Heuristic:17969.222877935514
125
LB:17758.73820191034
gap:0.003969284413914919
upper bound by Heuristic:17829.227684665977
126
LB:17759.10401655523
gap:0.003948603941132249
upper bound by Heuristic:17969.222877935514
127
LB:17759.1769062776
gap:0.003944483393462615
upper bound by Heuristic:17829.227684665977
128
LB:17759.58736005722
gap:0.003921280556629618
upper bound by Heuristic:17829.227684665977
129
LB:17759.678597934388
gap:0.0039161230507672885
upper bound by Heuristic:17969.222877935514
130
LB:17760.020878470477
gap:0.0038967750471169418
upper bo

...

LB:26917.58552915161
gap:0.0006239460470634057
upper bound by Heuristic:29426.977380899847
54
LB:26917.674610816623
gap:0.0006206345705538248
upper bound by Heuristic:26934.380650239014
55
LB:26917.719041566168
gap:0.0006189829326592492
upper bound by Heuristic:26934.380650239014
56
LB:26917.951985112562
gap:0.0006103237399166963
upper bound by Heuristic:27338.50905685376
57
LB:26918.068182030358
gap:0.0006060044167488229
upper bound by Heuristic:26934.380650239014
58
LB:26918.00901753189
gap:0.0006060044167488229
upper bound by Heuristic:26934.380650239014
59
LB:26918.200872602683
gap:0.0006010720297729386
upper bound by Heuristic:26934.380650239014
60
LB:26918.25977458401
gap:0.0005988825351267784
upper bound by Heuristic:26934.380650239014
61
LB:26918.277251705505
gap:0.0005982328803188395
upper bound by Heuristic:27338.50905685376
62
LB:26918.296138650825
gap:0.0005975308208714698
upper bound by Heuristic:26934.380650239014
63
LB:26918.337253282858
gap:0.0005960025244204029
upper b

LB:26917.64885747881
gap:0.0008072553078543855
upper bound by Heuristic:26939.37827239397
28
LB:26921.203119759615
gap:0.0006751240854096142
upper bound by Heuristic:26939.37827239397
29
LB:26924.51769719705
gap:0.000551934685109178
upper bound by Heuristic:26939.37827239397
30
LB:26927.022407476958
gap:0.00045886488041770534
upper bound by Heuristic:26939.37827239397
31
LB:26929.1492064518
gap:0.00037985106264403786
upper bound by Heuristic:26939.37827239397
32
LB:26930.99920419596
gap:0.0003111309808625454
upper bound by Heuristic:26939.37827239397
33
LB:26932.022416777527
gap:0.0002731267449064715
upper bound by Heuristic:27707.82917627275
34
LB:26932.562901329242
gap:0.00025305319399778126
upper bound by Heuristic:26939.37827239397
35
LB:26933.146063290686
gap:0.000231395511264768
upper bound by Heuristic:27707.82917627275
36
LB:26933.592585142793
gap:0.00021481305298900566
upper bound by Heuristic:26939.37827239397
37
LB:26934.31460249755
gap:0.00018800069618068075
upper bound by 

LB:1471.617462618164
gap:0.007377551053505978
upper bound by Heuristic:1482.4743955798604
23
LB:1472.4127621884663
gap:0.006833432614669332
upper bound by Heuristic:1482.4743955798604
24
LB:1472.6853213242612
gap:0.006647091618185413
upper bound by Heuristic:1615.4069429641338
25
LB:1472.9736371862211
gap:0.006450053248602765
upper bound by Heuristic:1521.3965794471976
26
LB:1473.0593897435706
gap:0.006391463848534121
upper bound by Heuristic:1515.3162159914023
27
LB:1473.356337271807
gap:0.006188630732017726
upper bound by Heuristic:1521.3965794471976
28
LB:1473.5595052776453
gap:0.006049901799205165
upper bound by Heuristic:1515.3162159914023
29
LB:1473.719488842673
gap:0.005940687358394595
upper bound by Heuristic:1521.3965794471976
30
LB:1473.8410465546663
gap:0.00585772057670391
upper bound by Heuristic:1614.014244417754
31
LB:1473.8026749091127
gap:0.00585772057670391
upper bound by Heuristic:1570.6912238599243
32
LB:1474.0810664355702
gap:0.005693939997876671
upper bound by Heur

LB:252.03128847691227
gap:0.018540121793184584
upper bound by Heuristic:264.0532853311205
58
LB:252.03221687223785
gap:0.018536369860595443
upper bound by Heuristic:265.92826236380824
59
LB:252.03541445510828
gap:0.018523447651007496
upper bound by Heuristic:264.0532853311205
60
LB:252.03603372093437
gap:0.018520945085184315
upper bound by Heuristic:265.92826236380824
61
LB:252.0388418106388
gap:0.010714461665231498
upper bound by Heuristic:254.73930231936822
62
LB:252.0419131367299
gap:0.010702145326023671
upper bound by Heuristic:266.24850776750105
63
LB:252.04474626386423
gap:0.010690784455721521
upper bound by Heuristic:265.5483962965203
64
LB:252.04504904993377
gap:0.01068957029543034
upper bound by Heuristic:257.9986444739728
65
LB:252.04722984739283
gap:0.010680825469120863
upper bound by Heuristic:263.76335099331976
66
LB:252.04879422593578
gap:0.01067455252739943
upper bound by Heuristic:266.24850776750105
67
LB:252.0481084226971
gap:0.01067455252739943
upper bound by Heuristi

LB:261.16275640236967
gap:0.007773707464296692
upper bound by Heuristic:263.6367948137892
77
LB:261.16275717436764
gap:0.007773704485314276
upper bound by Heuristic:271.6428805057984
78
LB:261.1621563823605
gap:0.007773704485314276
upper bound by Heuristic:271.64646729886704
79
0:02:50.364498


In [11]:
results_df = pd.DataFrame(results, columns = ['Best_LB', 'Best_UB','Iter','Time']) 
results_df.to_csv('MV2_new_new_lambda_without_p_revised_beta_3_heuris_results_v_22.csv')

In [901]:
min(z_UB)

262.4383878963377

In [69]:
(min(z_UB)-max(z_LB))/max(z_LB)

8.7169170629282e-05

In [80]:
type(ddfij)

pandas.core.frame.DataFrame